In [ ]:
random_seed = 42

import torch
import random
import numpy as np

random.seed(random_seed)

np.random.seed(random_seed)
torch.manual_seed(random_seed)
# Включаем куду
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


## Погружение в Natural Language Proccessing

Так как обработкой языка занимались давно, а нейронные сети пытаются вставить везде лишь лет 10, то прежде чем решать задачи с помощью NN хотелось бы попробовать решить ее классическими методами. 


В данном задании мы будем всеми силами пытаться построить языковую модель (language model). 


#### Что это за модель? 

Тут везде почему-то говорится про какую-то неведомую модель. Уточню, вообще говоря, модель (в статистическом смысле, конечно) является математическим представлением процесса. Почти всегда модели являются приближением процесса. Для этого есть несколько причин, но два наиболее важных:
1. Обычно мы наблюдаем только процесс ограниченное количество раз
2. Модель может быть исключительно сложной, поэтому мы ее упрощаем

Вот что обычно делает модель: она описывает, как моделируемый процесс __создает данные__. В нашем случае моделируемым явлением является человеческий язык. Языковая модель дает нам способ генерации человеческого языка. Эти модели обычно составлены из вероятностных распределений.

Модель построена путем наблюдения некоторых образцов, сгенерированных явлением, которое должно быть смоделировано. Таким же образом, языковая модель строится путем наблюдения за некоторым текстом.

### Bag Of Words

Попробуем закодить для настоящего корпуса собраний сочинений Федора Михайловича Достоевского

In [ ]:
! wget https://raw.githubusercontent.com/DLSchool/dlschool_old/master/materials/homeworks/hw09/dostoevsky.txt

--2020-05-30 16:56:24--  https://raw.githubusercontent.com/DLSchool/dlschool_old/master/materials/homeworks/hw09/dostoevsky.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20406049 (19M) [text/plain]
Saving to: ‘dostoevsky.txt’

dostoevsky.txt      100%[===================>]  19.46M  --.-KB/s    in 0.09s   

2020-05-30 16:56:33 (205 MB/s) - ‘dostoevsky.txt’ saved [20406049/20406049]



In [ ]:
import io

with io.open('dostoevsky.txt', 'r',encoding='utf8') as f:
    text = f.read().replace(u'\xa0', u' ').replace(u'\ufeff','')

In [ ]:
import nltk
from nltk import word_tokenize
nltk.download('punkt')
words = word_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1. Сколько слов и предложений в датасете?


In [ ]:
from nltk import sent_tokenize

sents_full = sent_tokenize(text)
sents = [word_tokenize(s) for s in sents_full]

In [ ]:
print('Кол-во предложений: ', len(sents_full))
print('Кол-во слов: ', len(words))

Кол-во предложений:  123548
Кол-во слов:  2305025


In [ ]:
from nltk.corpus import reuters
from collections import Counter
 
counts = Counter(words)
total_count = len(words)

# 20 наиболее популярных слов 
print(counts.most_common(n=20))
 

# Частота в данном контексте является вероятностью появления слова (независимой)
# Поэтому проверяем основное свойство - суммирование  в 1

for word in counts:
    counts[word] /= float(total_count)

print(round(sum(counts.values()),3))  # 1.0


print(counts.most_common(n=20))

[(',', 265751), ('.', 85110), ('и', 80477), ('—', 53930), ('не', 39621), ('в', 39218), ('что', 34282), ('я', 25127), ('!', 22568), ('на', 20863), ('с', 19459), ('?', 18314), ('он', 17170), ('как', 14967), (';', 14015), ('это', 12030), ('его', 11939), ('а', 11840), ('же', 11689), ('все', 10566)]
1.0
[(',', 0.11529202503226646), ('.', 0.03692367761737942), ('и', 0.03491372110931552), ('—', 0.02339670936323901), ('не', 0.01718896758169651), ('в', 0.017014132167763908), ('что', 0.01487272372317003), ('я', 0.010900966366959143), ('!', 0.00979078318022581), ('на', 0.009051094890510949), ('с', 0.008441990867778007), ('?', 0.00794525005151788), ('он', 0.0074489430700317785), ('как', 0.0064932050628518125), (';', 0.0060801943579787635), ('это', 0.0052190323315365345), ('его', 0.00517955336710014), ('а', 0.005136603724471535), ('же', 0.005071094673593562), ('все', 0.004583898222362014)]


2. Каковы частоты для слов "бесы", '"семья", "брат"'.

In [ ]:
[round(i,5) for i in [counts['бесы'], counts['семья'], counts['брат']]]

[0.0, 1e-05, 0.00022]

In [ ]:
len(words)

2305025

In [ ]:
import random
 
# Код для генерации текста уже имеющейся моделью
new_text = []
 
for _ in range(30):
    accumulator = .0
    r = random.random()
    for word, freq in counts.items():
        accumulator += freq
 
        if accumulator >= r:
            new_text.append(word)
            break

print(' '.join(new_text))

ходила , . там кровь « поспешал , ? , — он , : тем человеку — под неудовольствием ! столь столе так в попроще у , , участвовать Чего


3. Генерируем текст, который состои из 10 слов с вероятностями появления от 0.85 до 0.95

In [ ]:
new_text = []
 
for _ in range(10):
    for word, freq in counts.items():
      if freq > (0.85/100000) and freq < (0.95/100000) and word not in new_text: # and word not in new_text and word not in string.punctuation:
          new_text.append(word)
          #print (word, freq)
          break

print(' '.join(new_text))

веселы темный чистая письму недостает тесно весела окнами ошиблись старым


Как вы можете видеть, это не самый выразительный фрагмент контента. Полученный текст следует только частотным правилам языка и не более того.


#### Биграммы,  триграммы и n-граммы

Одна идея, которая может помочь нам создать более качественный текст, состоит в том, чтобы убедиться, что новое слово, которое мы добавляем в последовательность, хорошо сочетается с уже существующими словами. Проверка правильности слова после 10 слов может немного переборщить. Мы можем упростить все, чтобы проблема была разумной. Давайте сделаем так, чтобы новое слово получилось после последнего слова в последовательности (модель биграмм) или двух последних слов (модель триграмм) (можно так долго продолжать, из эмпирических наблюдений при n=5 модель всегда выдает нормальный текст ) .

Некоторые быстрые магии NLTK (библиотека для nlp) для извлечения биграмм / триграмм:

In [ ]:
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
 
first_sentence = word_tokenize(sents_full[0])
print(first_sentence)

# Get the bigrams
print(list(bigrams(first_sentence)))

# Get the padded bigrams
print(list(bigrams(first_sentence, pad_left=True, pad_right=True)))
 
# Get the trigrams
print(list(trigrams(first_sentence)))
 
# Get the padded trigrams
print(list(trigrams(first_sentence, pad_left=True, pad_right=True)))

['Федор', 'Михайлович', 'Достоевский', 'Бедные', 'люди', 'Ох', 'уж', 'эти', 'мне', 'сказочники', '!']
[('Федор', 'Михайлович'), ('Михайлович', 'Достоевский'), ('Достоевский', 'Бедные'), ('Бедные', 'люди'), ('люди', 'Ох'), ('Ох', 'уж'), ('уж', 'эти'), ('эти', 'мне'), ('мне', 'сказочники'), ('сказочники', '!')]
[(None, 'Федор'), ('Федор', 'Михайлович'), ('Михайлович', 'Достоевский'), ('Достоевский', 'Бедные'), ('Бедные', 'люди'), ('люди', 'Ох'), ('Ох', 'уж'), ('уж', 'эти'), ('эти', 'мне'), ('мне', 'сказочники'), ('сказочники', '!'), ('!', None)]
[('Федор', 'Михайлович', 'Достоевский'), ('Михайлович', 'Достоевский', 'Бедные'), ('Достоевский', 'Бедные', 'люди'), ('Бедные', 'люди', 'Ох'), ('люди', 'Ох', 'уж'), ('Ох', 'уж', 'эти'), ('уж', 'эти', 'мне'), ('эти', 'мне', 'сказочники'), ('мне', 'сказочники', '!')]
[(None, None, 'Федор'), (None, 'Федор', 'Михайлович'), ('Федор', 'Михайлович', 'Достоевский'), ('Михайлович', 'Достоевский', 'Бедные'), ('Достоевский', 'Бедные', 'люди'), ('Бедные', 'л

Теперь можно и модель над этими биграммами построить. 

In [ ]:
model = defaultdict(lambda: defaultdict(lambda: 0))
 
for sentence in sents:
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
        
# следует теперь привести количество встречаемостей к частотам
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [ ]:
# следует теперь привести количество встречаемостей к частотам
import itertools
for w1_w2 in itertools.islice(model,1):
    total_count = float(sum(model[w1_w2].values()))
    print (w1_w2, total_count)
    for w3 in model[w1_w2]:
      print (w3)
      print (model[w1_w2][w3])     
      #model[w1_w2][w3] /= total_count

In [ ]:
w1_w2 = ('Михайлович', 'Достоевский')
total_count = float(sum(model[w1_w2].values()))
print (total_count)
for w3 in model[w1_w2]:
  print (w3)
  print (model[w1_w2][w3])
  print(model[w1_w2][w3] / total_count)
  model[w1_w2][w3] /= total_count

model[w1_w2] 
#model[('чтоб', 'еще')]

4. Какова вероятность слова $w_3$ при условии $w_1, w_2$

Берем соответсвующие им биграммы: {[чтоб еще], [мухи, сочинили], [прямо, по]}

In [ ]:
for w3 in ['вам', 'слона', 'коридорчику']:
  for w1_w2 in [('чтоб', 'еще'), ('мухи', 'сочинили'), ('прямо', 'по')]:
    print('Вероятность "', w3, '" при', w1_w2, ': ', model[w1_w2][w3])

Вероятность " вам " при ('чтоб', 'еще') :  0
Вероятность " вам " при ('мухи', 'сочинили') :  0
Вероятность " вам " при ('прямо', 'по') :  0
Вероятность " слона " при ('чтоб', 'еще') :  0
Вероятность " слона " при ('мухи', 'сочинили') :  1.0
Вероятность " слона " при ('прямо', 'по') :  0
Вероятность " коридорчику " при ('чтоб', 'еще') :  0
Вероятность " коридорчику " при ('мухи', 'сочинили') :  0
Вероятность " коридорчику " при ('прямо', 'по') :  0.14285714285714285


Можете поискать еще интересных сочитаний в модели. Например найти самое часто встречаемое в тексте выражение. Или попробовать использовать пятиграммную модель. Это довольно интересно. 

Ниже, наконец представлен код для нашей генеративной модели. Рекомендую с ним так же поиграться.

In [ ]:
new_text = [None, None]
 
sentence_finished = False
 
while not sentence_finished:
    r = random.random()
    accumulator = .0
    for word in model[tuple(new_text[-2:])].keys():
        accumulator += model[tuple(new_text[-2:])][word]
        if accumulator >= r:
            new_text.append(word)
            break
 
    if new_text[-2:] == [None, None]:
        sentence_finished = True

print(' '.join([t for t in new_text if t]))

У ней ведь был совершенно пьян .


In [ ]:
start = 'Я тогда это говорил'

new_text = word_tokenize(start)
 
sentence_finished = False
 
while not sentence_finished:
    r = random.random()
    accumulator = .0
    for word in model[tuple(new_text[-2:])].keys():
        accumulator += model[tuple(new_text[-2:])][word]
        if accumulator >= r:
            new_text.append(word)
            break
 
    if new_text[-2:] == [None, None]:
        sentence_finished = True

print(' '.join([t for t in new_text if t]))


Я тогда это говорил , что к Кузьме Кузьмичу , к письмоводителю , а все-таки ш-дик : ты видишь , мой друг , до свидания .


5. Функция для генерации текста по итоговой модели

In [ ]:
def gen_text():
  new_text = [None, None]
  
  sentence_finished = False
  
  while not sentence_finished:
      r = random.random()
      accumulator = .0
      for word in model[tuple(new_text[-2:])].keys():
          accumulator += model[tuple(new_text[-2:])][word]
          if accumulator >= r:
              new_text.append(word)
              break
  
      if new_text[-2:] == [None, None]:
          sentence_finished = True

  fin = ' '.join([t for t in new_text if t])
  #print(fin)
  return fin

Генерируем текст 10 раз и находим самые частые символы

In [ ]:
all_texts = ''
for i in range(10):
  all_texts += gen_text()
print(all_texts)
print ('5 top characters:', Counter(all_texts).most_common(5))

Наш отпрыск , назад тому полчаса , но и то , что мог ; возлюбив , он уже не направиться к князю Льву Николаевичу , — а это много значит привычка !Он остановился и , наконец , после обеда в семь часов утра к нему еще раз придется развернуть мою рукопись и стали все угождать , стали смеяться : « Эх , барин чаю просит , — прервал Коровкин .Я у вас вовсе не такой… чудак , — закричал хмельной Разумихин , как в европейских литературах были громадной величины художественные гении – Шекспиры , Сервантесы , Шиллеры .— Ба !У Амалии Ивановны .Петр же Ильич , все же произошло затем ?А долго вы изволили тогда приходить , может быть , необходим стал и взял шляпу , она вырывалась из занемевших в отчаянной мольбе защитить , ибо он , должно быть , тут не пошлость эгоизма и желания устроить свою карьеру , выбиться в люди , ну там о каких врагах говорите вы ?Это была тихая , нежная , ясная тишина , с беспокойством , с своей тоской и поминутно преувеличивала беду .С невинною целию обезоружить его либерали

На данном этапе моедль уже может генерировать более осмысленный текст. Кроме того существует множество статистических методом, которые могают сгладить распределение слов, бороться с появление новых n-грамм. И такие модели до сих пор могут спокойно соревноваться с нейронными решениями. 

А теперь время для нейросетей

## char-RNN in PyTorch


In [ ]:
import torch
import random
import numpy as np

import codecs
import io

from torch import nn
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Закодируем наш текст в цифры

In [ ]:
! wget https://raw.githubusercontent.com/DLSchool/dlschool_old/master/materials/homeworks/hw09/dostoevsky.txt
with io.open('dostoevsky.txt', 'r',encoding='utf8') as f:
    text = f.read().replace(u'\xa0', u' ').replace(u'\ufeff','')
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])
encoded.shape # Наш словарь получился очень большой

--2020-06-05 13:13:12--  https://raw.githubusercontent.com/DLSchool/dlschool_old/master/materials/homeworks/hw09/dostoevsky.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20406049 (19M) [text/plain]
Saving to: ‘dostoevsky.txt’

dostoevsky.txt      100%[===================>]  19.46M  29.2MB/s    in 0.7s    

2020-06-05 13:13:13 (29.2 MB/s) - ‘dostoevsky.txt’ saved [20406049/20406049]



(11321980,)

In [ ]:
len(chars)

175

#### Обработка данных

Будем использовать для представления букв one-hot вектора.

In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Инициализируем вектора 
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # заполним 1 в соответсвующем месте
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Приводим к нужному размеру
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot
one_hot_encode(np.array([[4, 0, 3, 2], [1, 1, 3, 3]]), 5)

array([[[0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]],

       [[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]]], dtype=float32)

Генератор мини-батчей. Каждая последовательность будет длины `n_steps`

In [ ]:
def get_batches(arr, n_seqs, n_steps):
    """
    Создание генератора, возвращающего минибатчи размера (n_seqs x seq_len) Numpy
    """    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y
#check
next(get_batches(encoded, 8, 10))

(array([[  9, 161, 142,   0,  85,  84, 149, 167, 168,  23],
        [140,   0, 162,  84,  96,   0, 167,  84, 142, 161],
        [ 85,  23,  33,   0, 162,  64, 167, 117,  72,  84],
        [ 72,  84,  96,  23,  17,   0,  84,  17, 167,  84],
        [161, 117, 117,   0,  84, 169, 140, 161,  85, 161],
        [ 23,  28,  84,  66,   0,  84,   4,  84, 140,  33],
        [ 33,  23,  96,  84, 140, 151,  85,  23, 129, 167],
        [ 84, 162,   0,  84, 162,   0,  17,  77,  64,   0]]),
 array([[161, 142,   0,  85,  84, 149, 167, 168,  23,  68],
        [  0, 162,  84,  96,   0, 167,  84, 142, 161, 117],
        [ 23,  33,   0, 162,  64, 167, 117,  72,  84,  37],
        [ 84,  96,  23,  17,   0,  84,  17, 167,  84, 110],
        [117, 117,   0,  84, 169, 140, 161,  85, 161, 117],
        [ 28,  84,  66,   0,  84,   4,  84, 140,  33,   4],
        [ 23,  96,  84, 140, 151,  85,  23, 129, 167,  17],
        [162,   0,  84, 162,   0,  17,  77,  64,   0,  72]]))

### Построение charRNN модели

Почему делаем именно модель над символами? Потому что это очень простая структура, которая может быть применена не только к текстам, но и к музыке. 

In [ ]:
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
  print('Training on GPU!')
else:
  print('Train on CPU')

Training on GPU!


In [ ]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.2, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.dropout = nn.Dropout(drop_prob) #<dropout>
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        # self.lstm = nn.GRU(len(self.chars), n_hidden, n_layers,
        #                    dropout=drop_prob, batch_first=True) 
        #<torch gru will be the best choice>
        self.fc = nn.Linear(n_hidden, len(self.chars)) #<linear>
        
        self.init_weights()
        
    def forward(self, x, hc):
        ''' Forward pass through the network '''
        # print (x.shape, hc.shape)
        x, h = self.lstm(x, hc)
        x = self.dropout(x)
        #
        x = x.contiguous().view(-1, self.n_hidden)
        #
        x = self.fc(x)
        return x, h
    
    def predict(self, char, h=None, top_k=None):
        """        
            Returns the predicted character and the hidden state.
        """
        
        if h is None:
            h = self.init_hidden(1)#.to(device)
        
        x = np.array([[self.char2int[char]]])
        x =  one_hot_encode(x, len(self.chars)) #<one-hot>
        
        inputs = torch.from_numpy(x).to(device)
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = nn.functional.softmax(out, dim=1) #<Get proba vector (softmax on last dimension)>
        if(torch.cuda.is_available()):
            p = p.cpu() # move to cpu
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.detach().numpy().squeeze()
        # Choose 1/k 
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        if (torch.cuda.is_available()):
            hidden = (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_().cuda()
            ,weight.new(self.n_layers, n_seqs, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_()
                      ,weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())
        return hidden        

Для проверки функционирования у нас есть функция `train` , которая позволит провести большое число экспериментов.

In [ ]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.0005, clip=3, val_frac=0.1, print_every=200):
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr) #<Optimizer : Adam is the best choice>
    criterion = nn.CrossEntropyLoss() #<CE> 
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if(torch.cuda.is_available()):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        for x, y in get_batches(data, n_seqs, n_steps):
            counter += 1
            
            # Кодируем данные и отправлячем
            x = one_hot_encode(x, n_chars)
            inputs = torch.from_numpy(x).to(device)
            targets = torch.from_numpy(y).to(device)
            
            #  замените на .copy будет работать стабильнее
            h = tuple([each.data for each in h])
            #tuple([each.data for each in h])
            #h.data.clone().detach()
            #torch.tensor(h.data, device=device) 

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            
            loss = criterion(output, targets.view(n_seqs*n_steps)) #.view(n_seqs*n_steps,-1)
            loss.backward()
            
            # clip grad norm может вам помочь
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                net.eval()
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    inputs = torch.from_numpy(x).to(device)
                    targets = torch.from_numpy(y).to(device)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h =tuple([each.data for each in val_h])  # val_h.clone().detach()
                    output, val_h = net(inputs, val_h) #net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(n_seqs*n_steps)) #.view(n_seqs*n_steps,-1)
                    val_losses.append(val_loss.item())
                   
                # Попробуем валидироваться таким способом
                prime = 'Дом'
                top_k = 4
                chars = [ch for ch in prime]
                vh = None
                for ch in prime:
                    char, vh = net.predict(ch, vh, top_k=top_k)
                for ii in range(20):
                    char, vh = net.predict(chars[-1], vh, top_k=top_k)
                    chars.append(char)
                    
                chars.append(char)
                print(' '.join(chars))

                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)),
                       "Validation Perplexity: {:.4f}".format(np.exp(np.mean(val_losses))))
    return net

## Время тренировки!

![a](https://www.apmpodcasts.org/wp-content/uploads/2015/06/adventure-time.jpg)

In [ ]:
if 'net' in locals():
    del net
num_hidden_units = 256 #<подберите оптимальное значение (помните, что это буквы)>
net = CharRNN(chars, n_hidden=num_hidden_units, n_layers=3).to(device)
print(net)

CharRNN(
  (dropout): Dropout(p=0.2, inplace=False)
  (lstm): LSTM(175, 256, num_layers=3, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=175, bias=True)
)


In [ ]:
%%time

train(net, encoded, epochs=20,
      n_seqs=64, n_steps=100, lr=0.001, 
      print_every=200)

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

model_save_name = 'LSTM-20epochs.net'
with open(f'/content/gdrive/My Drive/Char-RNN/{model_save_name}', 'wb') as f:
    torch.save(checkpoint, f)

Д о м е н   и   п р о м и   н а   в е р о м н н
Epoch: 1/20... Step: 200... Loss: 2.5683... Val Loss: 2.4901 Validation Perplexity: 12.0624
Д о м е н н о   с   т о г   с е м и т е л ь н н
Epoch: 1/20... Step: 400... Loss: 2.2078... Val Loss: 2.1223 Validation Perplexity: 8.3500
Д о м е н и е .   —   А   п р о ч е м ,   п о о
Epoch: 1/20... Step: 600... Loss: 2.0503... Val Loss: 1.9138 Validation Perplexity: 6.7787
Д о м а н н ы е   и с п р о г и д н о   и ,    
Epoch: 1/20... Step: 800... Loss: 1.8988... Val Loss: 1.7950 Validation Perplexity: 6.0195
Д о м а н н ы й   и   п о т о в о р н о   с л л
Epoch: 1/20... Step: 1000... Loss: 1.7885... Val Loss: 1.7042 Validation Perplexity: 5.4971
Д о м е н и я   в   п о м н ю ,   н а   с т р р
Epoch: 1/20... Step: 1200... Loss: 1.8199... Val Loss: 1.6472 Validation Perplexity: 5.1926
Д о м е н и я   с о с т о я л и с ь   и   в ы ы
Epoch: 1/20... Step: 1400... Loss: 1.7288... Val Loss: 1.6087 Validation Perplexity: 4.9963
Д о м и ,   в с е   в с

Целая часть итоговой перплексии: **3**

### Загрузка модели 

После обучения мы сохраним модель, чтобы мы могли загрузить ее позже, если понадобится. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметров скрытого слоя и текстовых символов.

In [ ]:
model_save_name = 'LSTM-20epochs.net'
with open(f'/content/gdrive/My Drive/Char-RNN/{model_save_name}', 'rb') as f:
    checkpoint = torch.load(f, map_location=torch.device('cpu'))
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

#check forward pass:
# char = chars[5]
# x = np.array([[char2int[char]]])
# x =  one_hot_encode(x, len(chars)) #<one-hot>
# loaded.forward(torch.from_numpy(x).to(device),
#                tuple([each.data for each in loaded.init_hidden(1)]))

<All keys matched successfully>

### Сэмплирование, Top K method

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным признакам. Делаем выборку, учитывая только некоторые вероятные символы $ K $, чтобы сеть не давала нам абсолютно абсурдные символы, вводя некоторый шум и случайность в выбранный текст.


In [ ]:
def sample(net, size, prime='The', top_k=None, cuda=False):
    net.eval()
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = None
    for ch in prime:
        char, h = net.predict(ch, h,top_k=top_k)
        
    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

Пример работы модели.

In [ ]:
loaded.eval()
print(sample(loaded, 100, prime='Путин', top_k=2, cuda=True))

Путинский и под конец в своем получении своей подлости и с необходимыми, но в таком случае она приняла его


Посмотрим на семлы случайной модели.

In [ ]:
print(sample(CharRNN(chars, n_hidden=256, n_layers=2).to(device), 
             20, prime='Путин ', top_k=5, cuda=True))

Путин ЦVМЦ»ШЦ»VVШМШШ»ЦМVШШМ


Разница видна, значит мы все сделали правильно.

In [ ]:
print(sample(loaded, 1000, top_k=3, prime="Раз два"))

Раз два рубля на минуту, например, через них, в каком странной вере не может быть, чтоб ответил в темноте и потому принимался в нем на себя. Но теперь они становилось с того часа на дому, и все время не совсем не вынесла и выставилась в корнести и послышались на половине своего последнего вопроса, но на стуле и поступила в комнату, и отвечают на нее вопросы. Она подала ему всем восторгом. Вот тогда все выходят. В самом деле, когда он пришел к нему в сердце, но никак не мог послать на мою кровь в карман в подлость, и он в самом деле, и почему-то все это подозрилось на своей крайности. Она выслушала, как бы в сторону происхадивший в своих словах, и всегда, при всех отца от него наши слова старуха, и он просто страдал и в такой стороне несколько слов в странном смысле, что в том, что она сама пред нею совершенно понимаю и про себя начинала, не сознала вас в настоящих случаях, и перед вами представлялось в том, что в самых делах состоил в таком постель, и всегда принимается в свои два меся

### Итог

Варьирование параметров n_seqs, n_steps, n_layers и num_hidden_units не дало какого-то ощутимого падения итоговой перплексии, все сходилось к значению **3,5-3,8**. Кол-во эпох больше 20 особого прироста качества не принесло. Также не получилось обучить сеть с ячейкой GRU вместо LSTM - перплексия падала, но сэмплы модели давали предикты только на несколько возможных символов.